<a href="https://colab.research.google.com/github/jhackisneros/VISION_ARTIFICIAL/blob/main/09_robustness_tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# VISION_ARTIFICIAL - Notebook 09: Robustez de modelos


🔹 Importar librerías

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score
import os
from skimage.util import random_noise
from skimage import exposure

🔹 Cargar datos de test

In [ ]:
processed_dir = "../data/processed/"
x_test = np.load(os.path.join(processed_dir, "x_test.npy"))
y_test = np.load(os.path.join(processed_dir, "y_test.npy"))


🔹 Lista de modelos del ensemble

In [ ]:
ensemble_models = [
    "../models/ensemble/cnn_v1.keras",
    "../models/ensemble/cnn_v2.keras",
    "../models/ensemble/cnn_v3.keras"
]


🔹 Función para soft voting

In [ ]:
def ensemble_predict(models, x):
    pred_probs = np.zeros((x.shape[0], 10))
    for model in models:
        pred_probs += model.predict(x)
    pred_probs /= len(models)
    return np.argmax(pred_probs, axis=1)

🔹 Cargar modelos

In [ ]:
models = [load_model(path) for path in ensemble_models]

 🔹 Generar perturbaciones


In [ ]:
def add_gaussian_noise(images, var=0.01):
    noisy = np.array([random_noise(img, mode='gaussian', var=var) for img in images])
    return noisy

def adjust_brightness(images, gamma=0.8):
    bright = np.array([exposure.adjust_gamma(img, gamma) for img in images])
    return bright


🔹 Probar robustez con ruido

In [ ]:
x_test_noisy = add_gaussian_noise(x_test, var=0.02)
y_pred_noisy = ensemble_predict(models, x_test_noisy)
robust_acc_noisy = accuracy_score(np.argmax(y_test, axis=1), y_pred_noisy)
print(f"✅ Precisión con ruido gaussiano: {robust_acc_noisy*100:.2f}%")

🔹 Probar robustez con cambio de brillo

In [ ]:
x_test_bright = adjust_brightness(x_test, gamma=0.5)
y_pred_bright = ensemble_predict(models, x_test_bright)
robust_acc_bright = accuracy_score(np.argmax(y_test, axis=1), y_pred_bright)
print(f"✅ Precisión con cambio de brillo: {robust_acc_bright*100:.2f}%")

🔹 Guardar resultados

In [ ]:
os.makedirs("../experiments", exist_ok=True)
robustness_scores = {
    'accuracy_noisy': float(robust_acc_noisy),
    'accuracy_brightness': float(robust_acc_bright)
}

import json
with open("../experiments/robustness_scores.json", "w") as f:
    json.dump(robustness_scores, f, indent=4)

print("✅ Resultados de robustez guardados en 'experiments/robustness_scores.json'")